In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import glob

"""Carga de archivos DICOM"""
dicoms = []
directorio = 'F:/usuarios/alumno/Documentos/MFM/Mis Apuntes/Taller de Informatica/Presentar/Nueva carpeta/CT/*.dcm'

for fname in glob.glob(directorio, recursive=False):
    dicoms.append(pydicom.read_file(fname))


"""Ordeno según Slice Location"""    
def param_1(elem):
    return elem[0x20, 0x1041].value

dicoms.sort(key = param_1, reverse = True)


"""Matriz de datos multiplano"""
img_shape = list(dicoms[0].pixel_array.shape)
img_shape.append(len(dicoms))

matriz3D = np.zeros(img_shape)

for i, slice in enumerate(dicoms):
    corte = slice.pixel_array * slice.RescaleSlope + slice.RescaleIntercept
    matriz3D[:, :, i] = corte

    
"""Corte de planos"""
centro = 50
ancho = 400
v_min = centro - ancho/2
v_max = centro + ancho/2 

espaciado = slice.PixelSpacing
espesor = slice.SliceThickness
cor_aspect = espesor/espaciado[0]
fig_1 = plt.figure(1, figsize=(15,15))
a1 = fig_1.add_subplot(221)
a1.imshow(matriz3D[250, :, :].T, cmap='gray', vmin=v_min, vmax=v_max)
a1.set_aspect(cor_aspect)
a1.set_title('MPR CORONAL')

a2 = fig_1.add_subplot(222)
a2.imshow(matriz3D[:, 250, :].T, cmap='gray', vmin=v_min, vmax=v_max)
a2.set_aspect(cor_aspect)
a2.set_title('MPR SAGITAL')

a3 = fig_1.add_subplot(223) #Se elige plano en donde se aprecien pulmones y corazón
a3.imshow(matriz3D[:, :, 320], cmap='gray', vmin=v_min, vmax=v_max)
a3.set_title('MPR AXIAL')
plt.show()

fig_1.savefig("imagen.png", dpi=200)
fig_1.savefig("imagen.pdf")


"""Desarrollo de función"""
def imagen_x_corte(c_a, c_c, c_s, centro, ancho):
    """c_a = n° corte axial, c_c = n° corte coronal, c_s= n° corte sagital"""
    v_min = centro - ancho/2
    v_max = centro + ancho/2
    
    espaciado = slice.PixelSpacing
    espesor = slice.SliceThickness
    cor_aspect = espesor/espaciado[0]
    fig_1 = plt.figure(1, figsize=(15,15))
    a1 = fig_1.add_subplot(221)
    a1.imshow(matriz3D[c_c, :, :].T, cmap='gray', vmin=v_min, vmax=v_max)
    a1.set_aspect(cor_aspect)
    a1.set_title('MPR CORONAL')
    
    a2 = fig_1.add_subplot(222)
    a2.imshow(matriz3D[:, c_s, :].T, cmap='gray', vmin=v_min, vmax=v_max)
    a2.set_aspect(cor_aspect)
    a2.set_title('MPR SAGITAL')
    
    a3 = fig_1.add_subplot(223) #Se elige plano en donde se aprecien pulmones y corazón
    a3.imshow(matriz3D[:, :, c_a], cmap='gray', vmin=v_min, vmax=v_max)
    a3.set_title('MPR AXIAL')
    plt.show()
    
    fig_1.savefig("imagen.png", dpi=200)
    fig_1.savefig("imagen.pdf")
    

"""Hueso"""    
imagen_x_corte(250,250,250,200,100)


"""mAs en función de n° corte"""
mAs = []
for slice in dicoms:
    mAs.append(slice.XRayTubeCurrent)

x = np.linspace(0,len(dicoms)+1,len(dicoms))

fig_2 = plt.figure(1, figsize=[10, 10])
graf1 = fig_2.add_subplot(111)
graf1.grid()
graf1.plot(x, mAs)